In [731]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [732]:
import pandas as pd
from numpy import nan
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation

from sklearn import svm
from sklearn import decomposition
from sklearn.neighbors import KNeighborsClassifier


In [733]:
# Set the global default size of matplotlib figures
#plt.rc('figure', figsize=(10,5))

In [734]:
train_data = pd.read_csv('./Data/ML Data Collection.csv')

In [735]:
scalar = preprocessing.MinMaxScaler(feature_range=(0,1))

#Mapping priority list into numbers
priority_list = sorted(train_data['Priority'].unique())

float_list = [float(i) for i in range(0,len(priority_list))]
priority_mapping = dict(zip(priority_list,float_list))
train_data['priority_val'] = train_data['Priority'].map(priority_mapping)

train_data['priority_val'] = (scalar.fit_transform(train_data['priority_val'].reshape(-1, 1))).ravel([1])

In [736]:
# APT normalization
train_data['APT'] = (scalar.fit_transform(train_data['APT'].reshape(-1, 1))).ravel([1])

In [737]:
# TSLR normalization
train_data['TSLR'] = (scalar.fit_transform(train_data['TSLR'].reshape(-1, 1))).ravel([1])

In [738]:
# Check if 'Calls from Customer' needs to be normalized
floatCallsArray =  np.asarray([float(i) for i in train_data['Calls from Customer']])
train_data['Calls from Customer'] = (scalar.fit_transform(floatCallsArray.reshape(-1, 1))).ravel([1])

In [739]:
# Check if 'Number of Info to SAP from last TSLR' needs to be normalized
floatNumInfoArray =  np.asarray([float(i) for i in train_data['Number of Info to SAP from last TSLR']])
train_data['Number of Info to SAP from last TSLR'] = (scalar.fit_transform(floatNumInfoArray.reshape(-1, 1))).ravel([1])

In [740]:
# Check if 'Frequent changes in processor' needs to be normalized - might not be, need to check the whole data
floatFreqChangeArray =  np.asarray([float(i) for i in train_data['Frequent changes in processor']])
train_data['Frequent changes in processor'] = (scalar.fit_transform(floatFreqChangeArray.reshape(-1, 1))).ravel([1])

In [741]:
# Check if 'Number of Processors' needs to be normalized - might not be, need to check the whole data
floatNumProcArray =  np.asarray([float(i) for i in train_data['Number of Processors']])
train_data['Number of Processors'] = (scalar.fit_transform(floatNumProcArray.reshape(-1, 1))).ravel([1])

In [742]:
#Mapping callback list into numbers
#callback_list = sorted(train_data['Customer call back request'].unique())
#callback_mapping = dict(zip(callback_list,range(0,len(callback_list))))
#train_data['callback_val'] = train_data['Customer call back request'].map(callback_mapping)

In [743]:
#Mapping core business list into numbers
core_business_list = sorted(train_data['Core Business affected?'].unique())
core_business_list.remove(nan)
core_business_mapping = dict(zip(core_business_list,range(0,len(core_business_list))))
train_data['core_business_val'] = train_data['Core Business affected?'].map(core_business_mapping)

In [744]:
train_data['core_business_val'] = train_data['core_business_val'].fillna(train_data['core_business_val'].median())


In [745]:
#Mapping Go live list into numbers
go_live_list = sorted(train_data['Go live Affected?'].unique())
go_live_list.remove(nan)
go_live_mapping = dict(zip(go_live_list,range(0,len(go_live_list))))
train_data['go_live_val'] = train_data['Go live Affected?'].map(go_live_mapping)

In [746]:
train_data['go_live_val'] = train_data['go_live_val'].fillna(train_data['go_live_val'].median())

In [747]:
#Either normalize or remove Num of users affected

In [748]:
#Mapping Work around list into numbers
work_around_list = sorted(train_data['Work around available?'].unique())
work_around_list.remove(nan)
work_around_mapping = dict(zip(work_around_list,range(0,len(work_around_list))))
train_data['work_around_val'] = train_data['Work around available?'].map(work_around_mapping)

In [749]:
train_data['work_around_val'] = train_data['work_around_val'].fillna(train_data['work_around_val'].median())

In [750]:
#Mapping escalation states into numbers
escStages = sorted(train_data['Stage of Escalation'].unique())
states_mapping = dict(zip(escStages,range(0,len(escStages))))
train_data['escStages_val'] = train_data['Stage of Escalation'].map(states_mapping)

In [751]:
train_data = train_data.drop(['Stage of Escalation', 'Priority', 'Customer call back request', 'Core Business affected?',
                             'Go live Affected?', 'Work around available?'], axis=1)

In [752]:
train_data = train_data.drop(['Num of users affected', 'Business Impact mentioned?'], axis=1)

In [753]:
#removing to test
train_data = train_data.drop(['core_business_val','work_around_val'], axis=1)

In [761]:
#useful
train_data

,APT,Calls from Customer,TSLR,Number of Info to SAP from last TSLR,Frequent changes in processor,Number of Processors,priority_val,go_live_val,escStages_val
0,0.000000,0.052632,0.000000,0.000000,0.000,0.0,0.000000,1.0,0
1,0.068418,0.105263,0.046512,0.000000,0.125,0.2,0.000000,1.0,1
2,0.083690,0.105263,0.000000,0.058824,0.125,0.2,0.000000,1.0,2
3,0.086859,0.105263,0.000000,0.058824,0.125,0.2,0.000000,1.0,3
4,0.125038,0.105263,0.058140,0.117647,0.125,0.2,0.000000,1.0,4
5,0.140310,0.105263,0.081395,0.235294,0.125,0.2,0.000000,1.0,5
6,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.666667,1.0,0
7,0.117670,0.000000,0.032558,0.117647,0.125,0.0,0.000000,1.0,1
8,0.132941,0.000000,0.032558,0.117647,0.125,0.0,0.000000,1.0,2
9,0.045938,0.000000,0.000465,0.058824,0.000,0.0,0.000000,1.0,3


In [755]:
train_values = train_data.values
X = train_values[:,:-1]
Y = train_values[:,-1:]

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,Y, test_size=0.33, random_state=0)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(229L, 8L) (113L, 8L) (229L, 1L) (113L, 1L)


In [756]:
#rf = RandomForestClassifier(n_estimators=800,max_depth=50)
rf = RandomForestClassifier(n_estimators=870, min_samples_leaf =8, n_jobs = -1, max_depth=12)
rf.fit(X_train,y_train.ravel([1]))

predicted_Y = rf.predict(X_test)


In [757]:
#pca = decomposition.RandomizedPCA(n_components=3)
#pca.fit(X_train)
#X_train_pca = pca.transform(X_train)
#x_test_pca = pca.transform(X_test)

#svc = svm.SVC(kernel='rbf', C=1e9, gamma=1e3)
#svc.fit(X_train,y_train.ravel([1]))
#svc.fit(X_train_pca,y_train.ravel([1]))

#predicted_svc = svc.predict(x_test_pca)

In [758]:
#useful
print rf.score(X_train,y_train)
y_test_ravel = y_test.ravel([1])
#print y_test_ravel,'\n',predicted_Y

#print 'x'*25

#print svc.score(X_train_pca,y_train)
#y_test_ravel = y_test.ravel([1])
#print y_test_ravel,'\n',predicted_svc

print(metrics.f1_score(y_test_ravel, predicted_Y))
#print(metrics.f1_score(y_test_ravel, predicted_svc))


0.471615720524
0.255352631546


C:\Prajeen\_Workstation\Anaconda\lib\site-packages\sklearn\metrics\classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


In [759]:

print metrics.confusion_matrix(predicted_Y,y_test_ravel)
#print metrics.confusion_matrix(predicted_kn,y_test_ravel)

[[20  9  7  5  2  0]
 [ 1  1  7  3  2  0]
 [ 1  3  0  1  0  0]
 [ 0  1  1  2  1  0]
 [ 0  3  1  3  4  3]
 [ 0  7  4  3  7 11]]


In [760]:
print metrics.classification_report(predicted_Y,y_test_ravel)
#print metrics.classification_report(predicted_kn,y_test_ravel)

             precision    recall  f1-score   support

        0.0       0.91      0.47      0.62        43
        1.0       0.04      0.07      0.05        14
        2.0       0.00      0.00      0.00         5
        3.0       0.12      0.40      0.18         5
        4.0       0.25      0.29      0.27        14
        5.0       0.79      0.34      0.48        32

avg / total       0.61      0.34      0.42       113



In [218]:
#print to the file
fileName = 'algoResult.txt'
#initialRead =  open(fileName)
#print initialRead.read()

file =  open(fileName, 'w')
file.truncate()

for i in range(predicted_Y.size):
    file.write(repr(int(predicted_Y[i]))+','+'\n')
    
file.close()

In [929]:
#Notes
# do confusion_matrix and classification_report from 4.3 -- done
# implement train_test_split from 4.3 , check also 6.1 -- done
# implement 4.3 Exercise: Model Selection via Validation -- done
# implement 5.1 PCA and SVM (also from 7.1) - done

# implement GridSearchCV from 6.2 and 7.2
